# 🔄 Data Preprocessing & Augmentation

This notebook preprocesses the collected hand landmark data and prepares it for model training.

## Objectives
- Load collected landmark data
- Explore and visualize the dataset
- Normalize and standardize features
- Apply data augmentation techniques
- Encode labels
- Split data into train/validation/test sets
- Save processed data

---

## 1. Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils import shuffle
import pickle
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)

print("✅ Libraries imported successfully")

## 2. Load Raw Data

In [ ]:
# Load data
DATA_DIR = 'data/raw'
PROCESSED_DIR = 'data/processed'
os.makedirs(PROCESSED_DIR, exist_ok=True)

# Load landmarks and labels
X = np.load(os.path.join(DATA_DIR, 'landmarks.npy'))
y = np.load(os.path.join(DATA_DIR, 'labels.npy'))

print("\n" + "="*60)
print("RAW DATA LOADED")
print("="*60)
print(f"Data shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Number of features: {X.shape[1]}")
print(f"Number of samples: {X.shape[0]}")
print(f"Unique classes: {np.unique(y)}")
print(f"Number of classes: {len(np.unique(y))}")
print("="*60)

## 3. Load Metadata

In [ ]:
# Load metadata if available
metadata_path = os.path.join(DATA_DIR, 'metadata.json')
if os.path.exists(metadata_path):
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    print("\nMetadata:")
    print(json.dumps(metadata, indent=2))
else:
    print("\n⚠️ No metadata file found")

## 4. Exploratory Data Analysis

In [ ]:
# Class distribution
unique, counts = np.unique(y, return_counts=True)
class_distribution = dict(zip(unique, counts))

print("\nClass Distribution:")
print("="*40)
for class_name, count in sorted(class_distribution.items()):
    percentage = (count / len(y)) * 100
    print(f"  {class_name:10s}: {count:4d} ({percentage:5.1f}%)")
print("="*40)

In [ ]:
# Visualize class distribution
plt.figure(figsize=(12, 6))
bars = plt.bar(unique, counts, color='steelblue', edgecolor='navy', linewidth=1.5)

# Add value labels
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.xlabel('Class', fontsize=12, fontweight='bold')
plt.ylabel('Number of Samples', fontsize=12, fontweight='bold')
plt.title('Class Distribution - Raw Data', fontsize=14, fontweight='bold')
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()
plt.savefig('outputs/visualizations/class_distribution_raw.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Feature statistics
print("\nFeature Statistics:")
print("="*60)
print(f"Min value: {X.min():.6f}")
print(f"Max value: {X.max():.6f}")
print(f"Mean value: {X.mean():.6f}")
print(f"Std deviation: {X.std():.6f}")
print(f"Median value: {np.median(X):.6f}")
print("="*60)

In [ ]:
# Visualize feature distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Sample a few features to visualize
sample_features = [0, 1, 2, 20]  # x, y, z of first landmark, and x of 7th landmark
feature_names = ['Landmark 0 - X', 'Landmark 0 - Y', 'Landmark 0 - Z', 'Landmark 7 - X']

for idx, (ax, feat_idx, feat_name) in enumerate(zip(axes.flat, sample_features, feature_names)):
    ax.hist(X[:, feat_idx], bins=50, color='skyblue', edgecolor='black', alpha=0.7)
    ax.set_xlabel('Value', fontsize=10)
    ax.set_ylabel('Frequency', fontsize=10)
    ax.set_title(feat_name, fontsize=11, fontweight='bold')
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/visualizations/feature_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Data Cleaning

In [ ]:
# Check for missing values
print("\nData Quality Check:")
print("="*60)

nan_count = np.isnan(X).sum()
inf_count = np.isinf(X).sum()

print(f"NaN values: {nan_count}")
print(f"Inf values: {inf_count}")

if nan_count > 0 or inf_count > 0:
    print("\n⚠️ Cleaning data...")
    # Remove rows with NaN or Inf
    valid_indices = ~(np.isnan(X).any(axis=1) | np.isinf(X).any(axis=1))
    X = X[valid_indices]
    y = y[valid_indices]
    print(f"✅ Removed {(~valid_indices).sum()} invalid samples")
    print(f"   Remaining samples: {X.shape[0]}")
else:
    print("✅ No missing or infinite values found")

print("="*60)

## 6. Label Encoding

In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("\nLabel Encoding:")
print("="*60)
print(f"Original labels: {label_encoder.classes_}")
print(f"Encoded range: 0 to {y_encoded.max()}")
print("\nMapping:")
for idx, class_name in enumerate(label_encoder.classes_):
    print(f"  {class_name} -> {idx}")
print("="*60)

# Save label encoder
os.makedirs('models', exist_ok=True)
with open('models/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)
print("\n✅ Label encoder saved to models/label_encoder.pkl")

## 7. Feature Normalization

In [ ]:
# Normalize features using StandardScaler
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

print("\nFeature Normalization:")
print("="*60)
print("Before normalization:")
print(f"  Mean: {X.mean():.6f}")
print(f"  Std:  {X.std():.6f}")
print(f"  Min:  {X.min():.6f}")
print(f"  Max:  {X.max():.6f}")

print("\nAfter normalization:")
print(f"  Mean: {X_normalized.mean():.6f}")
print(f"  Std:  {X_normalized.std():.6f}")
print(f"  Min:  {X_normalized.min():.6f}")
print(f"  Max:  {X_normalized.max():.6f}")
print("="*60)

# Save scaler
with open('models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print("\n✅ Scaler saved to models/scaler.pkl")

## 8. Data Augmentation

In [ ]:
def augment_data(X, y, augmentation_factor=2):
    """
    Augment hand landmark data with noise and transformations.
    
    Args:
        X: Feature array
        y: Label array
        augmentation_factor: Number of augmented copies per sample
    
    Returns:
        Augmented X and y arrays
    """
    X_augmented = [X]
    y_augmented = [y]
    
    for i in range(augmentation_factor):
        # Add Gaussian noise
        noise = np.random.normal(0, 0.02, X.shape)
        X_noisy = X + noise
        
        # Random scaling
        scale_factor = np.random.uniform(0.95, 1.05)
        X_scaled = X * scale_factor
        
        X_augmented.extend([X_noisy, X_scaled])
        y_augmented.extend([y, y])
    
    return np.vstack(X_augmented), np.concatenate(y_augmented)

# Apply augmentation
print("\nData Augmentation:")
print("="*60)
print(f"Original dataset size: {X_normalized.shape[0]}")

X_augmented, y_augmented = augment_data(X_normalized, y_encoded, augmentation_factor=1)

print(f"Augmented dataset size: {X_augmented.shape[0]}")
print(f"Augmentation ratio: {X_augmented.shape[0] / X_normalized.shape[0]:.1f}x")
print("="*60)

## 9. Train/Validation/Test Split

In [ ]:
# Shuffle data
X_augmented, y_augmented = shuffle(X_augmented, y_augmented, random_state=42)

# Split: 70% train, 15% validation, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(
    X_augmented, y_augmented, 
    test_size=0.3, 
    random_state=42, 
    stratify=y_augmented
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, 
    test_size=0.5, 
    random_state=42, 
    stratify=y_temp
)

print("\nData Split:")
print("="*60)
print(f"Training set:   {X_train.shape[0]:5d} samples ({X_train.shape[0]/X_augmented.shape[0]*100:.1f}%)")
print(f"Validation set: {X_val.shape[0]:5d} samples ({X_val.shape[0]/X_augmented.shape[0]*100:.1f}%)")
print(f"Test set:       {X_test.shape[0]:5d} samples ({X_test.shape[0]/X_augmented.shape[0]*100:.1f}%)")
print(f"Total:          {X_augmented.shape[0]:5d} samples")
print("="*60)

In [ ]:
# Verify stratification
print("\nClass Distribution Verification:")
print("="*60)
print(f"{'Class':<10} {'Train':<10} {'Val':<10} {'Test':<10}")
print("-" * 60)

for class_idx, class_name in enumerate(label_encoder.classes_):
    train_count = (y_train == class_idx).sum()
    val_count = (y_val == class_idx).sum()
    test_count = (y_test == class_idx).sum()
    print(f"{class_name:<10} {train_count:<10} {val_count:<10} {test_count:<10}")

print("="*60)

## 10. Save Processed Data

In [ ]:
# Save processed data
print("\nSaving processed data...")

np.save(os.path.join(PROCESSED_DIR, 'X_train.npy'), X_train)
np.save(os.path.join(PROCESSED_DIR, 'X_val.npy'), X_val)
np.save(os.path.join(PROCESSED_DIR, 'X_test.npy'), X_test)
np.save(os.path.join(PROCESSED_DIR, 'y_train.npy'), y_train)
np.save(os.path.join(PROCESSED_DIR, 'y_val.npy'), y_val)
np.save(os.path.join(PROCESSED_DIR, 'y_test.npy'), y_test)

print("✅ Processed data saved to data/processed/")
print("\nFiles created:")
print("  - X_train.npy, y_train.npy")
print("  - X_val.npy, y_val.npy")
print("  - X_test.npy, y_test.npy")

## 11. Create Processing Metadata

In [ ]:
# Create preprocessing metadata
preprocessing_metadata = {
    'preprocessing_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'original_samples': X.shape[0],
    'augmented_samples': X_augmented.shape[0],
    'train_samples': X_train.shape[0],
    'val_samples': X_val.shape[0],
    'test_samples': X_test.shape[0],
    'num_features': X_train.shape[1],
    'num_classes': len(label_encoder.classes_),
    'classes': label_encoder.classes_.tolist(),
    'normalization': 'StandardScaler',
    'augmentation_factor': X_augmented.shape[0] / X.shape[0],
    'train_split': 0.7,
    'val_split': 0.15,
    'test_split': 0.15,
    'random_seed': 42
}

# Save metadata
with open(os.path.join(PROCESSED_DIR, 'preprocessing_metadata.json'), 'w') as f:
    json.dump(preprocessing_metadata, f, indent=4)

print("\n✅ Preprocessing metadata saved!")
print("\nMetadata:")
print(json.dumps(preprocessing_metadata, indent=2))

## 12. Summary Visualization

In [ ]:
# Create summary visualization
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Dataset sizes
sizes = [X_train.shape[0], X_val.shape[0], X_test.shape[0]]
labels_pie = ['Train (70%)', 'Validation (15%)', 'Test (15%)']
colors = ['#66b3ff', '#99ff99', '#ffcc99']

axes[0].pie(sizes, labels=labels_pie, colors=colors, autopct='%1.1f%%',
           startangle=90, textprops={'fontsize': 11, 'fontweight': 'bold'})
axes[0].set_title('Data Split Distribution', fontsize=13, fontweight='bold')

# Class distribution in training set
train_class_counts = []
for class_idx in range(len(label_encoder.classes_)):
    train_class_counts.append((y_train == class_idx).sum())

axes[1].bar(label_encoder.classes_, train_class_counts, 
           color='steelblue', edgecolor='navy', linewidth=1.5)
axes[1].set_xlabel('Class', fontsize=11, fontweight='bold')
axes[1].set_ylabel('Number of Samples', fontsize=11, fontweight='bold')
axes[1].set_title('Training Set Class Distribution', fontsize=13, fontweight='bold')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/visualizations/preprocessing_summary.png', dpi=300, bbox_inches='tight')
plt.show()

---

## 🎯 Summary

Data preprocessing completed successfully!

### What Was Done:
- ✅ Loaded and cleaned raw data
- ✅ Encoded labels and saved encoder
- ✅ Normalized features using StandardScaler
- ✅ Applied data augmentation
- ✅ Split data into train/val/test sets (70/15/15)
- ✅ Saved processed data and metadata

### What's Next?
Proceed to **04_model_training.ipynb** to:
- Build the deep learning model architecture
- Train the model on processed data
- Monitor training with callbacks
- Save the best model

---